In [1]:
import pandas as pd
from difflib import SequenceMatcher, get_close_matches
import json
from urllib import parse , request
import ast # Para evaluar la string como diccionarios
from datetime import datetime
import numpy as np
import time
from IPython.display import HTML, display
import tabulate
import re

In [17]:
path = './data/IDE calles_localidad_departamento_24-7-2020.csv'
path_prueba1 = './data/direcciones_codificadas.csv'
path_prueba2 = './data/montevideo.csv'
base_INE = './data/direcciones_INE.csv'
localidades_INE = './data/localidades_INE.csv'
codificador = pd.read_csv(path)

NameError: name 'df' is not defined

Para el nomeclator. evitar ingresar calle "sin nombre".  Los nombres de las calles que contienen un numero debe de ser en digito

Notas. El principal problema que estoy encontrando es que las calles dentro mismo del nomenclator de INE se encuentran duplicadadas. Un 19% del dataset son entradas con idcalle duplicado. Puede ser que sean la misma calle que continua en otra localidad, ej: avenidad de las instrucciones que comunica toledo chico y montevideo. 
Un 6.5 % de las calles son duplicadas 721 nombres que generan 1888, es decir, se encuentran en la misma localidad y departamento y tienen el mismo nombre. Algunas son calles reales, ej: Progreso tiene 2 calles "Los Olivos" en distintas puntas del pueblo. Montevideo tiene 2 calles "Treinta y tres" (9231, 18853). Otras parecen ser errores de ingresado, ej: 'MACIEL, FRANCISCO ANTOÑO' en tacuarembo aparece con dos id distintos pero es la misma calle. La calle CLITERMNESTRA tiene 3 id distintas en 3 localidades diferentes (2 por cada localidad) lo que generan 6 anotaciones. 

Tambien encontre calles que faltan tanto en el nomenclator de INE como en la API, por ejemplo, 'CARLOS TIZZE' en AIGUA. 

Los calles con nombre de fecha tienen criterios distintos, ejemplo dieciocho de julio, ocho de octubre,veintiuno de setiembre, veinticeis de marzo Vs. 12 DE FEBRERO, 17 DE FEBRERO, ETC. OPCION: ADECUAR EL CODIFICADOR A UN FORMATO UNICO DE FECHAS, QUIZAS LLEVARLAS A NOMBRE?

HAY CALLES EN EL NOMENCLATOR Y API CON NUMERO ESCRITO TEXTUAL 'UNO', 'TRES' Y CON DIGITO '2', '4'

Como se definen dos secuencias iguales. Hablar de SequenceMatcher. El cutoff es de 0.6

In [ ]:
def suggest_dep(string):
    """Suggest a departamento """
    departamentos =(["artigas",
                    "canelones",
                    "cerro largo",
                    "colonia",
                    "durazno",
                    "flores",
                    "florida",
                    "lavalleja",
                    "maldonado",
                    "montevideo",
                    "paysandú",
                    "río negro",
                    "rivera",
                    "rocha",
                    "salto",
                    "san josé",
                    "soriano",
                    "tacuarembó",
                    "treinta y tres"])
    return get_close_matches(string.lower(),departamentos,n=2,cutoff=0.85)[0]

In [ ]:
def process_street(street):
    forbidden_char = ["Nº", "S/N",",","."] #Remplaza caracteres que estorban en la busqueda
    non_street = [999] #Codigos para marcar que no es una calle
    special_streets = ["SENDA","CALLE"]
    
    
    if street in non_street: 
        return "",""
    
    try:
        street = street.upper() # Se fija si la calle contiene texto, Si es la calle es un numero lo devuelve como string
    except: 
        return str(street) , "" #LA API de IDE acepta las calles con numero en formato str(numero)   str(numero) 
    
    for char in forbidden_char:
        street = street.replace(char,"")
    try:
        if not street.split()[-2] in special_streets: #Para ver si el antepenultima palabra no es numero, para que no parsee ej "Senda 8"
            number = int(street.split()[-1])
            street = ' '.join(street.split()[:-1])
        else:
            number = ""
    except Exception as e:
        number = ""
    return street, number

In [ ]:
def sugerir_calles(string,tryouts=3):
    """Sugiere calles a partir de una string"""
    query_args = {
        "entrada" : string, # La sugerencia a buscar
        "todos":False
    }
    root = 'https://direcciones.ide.uy/'
    URL = root + '/api/v0/geocode/SugerenciaCalleCompleta?'
    #r = request.Request(
    #        url=URL,
    #        data=parse.urlencode(query_args).encode('utf-8'),
    #    )
    
    while tryouts>0:
        try:
            r = request.urlopen(URL+parse.urlencode(query_args))
            data = r.read().decode('utf-8').replace('null','None')
            break
        except Exception as e:
            tryouts -= 1
            if tryouts == 0:
                print(e)
                return []
            time.sleep(0.5)
            
            
    if data=='[]':
        
        #print('No existe calle similar')
        return []
    else:
        return ast.literal_eval(data)

In [ ]:
def sugerir_localidad(localidad,departamento,limit=10,tryouts=3):
    """Sugiere una localidad a partir de una string y checkea contra un departamento devolviendo sugerencia e id"""

    
    departamentos =(["artigas",
                    "canelones",
                    "cerro largo",
                    "colonia",
                    "durazno",
                    "flores",
                    "florida",
                    "lavalleja",
                    "maldonado",
                    "montevideo",
                    "paysandú",
                    "río negro",
                    "rivera",
                    "rocha",
                    "salto",
                    "san josé",
                    "soriano",
                    "tacuarembó",
                    "treinta y tres"])
    
    candidato_departamento = get_close_matches(departamento.lower(),departamentos,n=limit,cutoff=0.6)[0] 
    
    string = localidad + ", "+candidato_departamento
    query_args = {
        "limit": 10,
        "q" : string, # La sugerencia a buscar
        "soloLocalidad":True
    }
    
    
    root = 'https://direcciones.ide.uy/'
    URL = root + 'api/v1/geocode/candidates?'
    while tryouts>0:
        #r = request.Request(
        #        url=URL,
        #        data=parse.urlencode(query_args).encode('utf-8'),
        #    )
        try:
            r = request.urlopen(URL+parse.urlencode(query_args))
            break
        except Exception as e:
            tryouts -= 1
            if tryouts ==0:
                print(e)
                return {}
            time.sleep(0.5)
    
    data = r.read().decode('utf-8')
    
    if data=='[]':
        #print('No existe localidad similar')
        return {}
    else:
        data = ast.literal_eval(data.replace('null','None')) 
    try:
        localidades = [x.get('address').lower().split(',')[0] for x in data]
        idlocalidades = [x.get('idLocalidad') for x in data]
        loc_suggested = get_close_matches(localidad.lower(),localidades,n=limit,cutoff=0.6)[0] #Para localidad el matcheo queda en 0.6
        idloc_suggested = idlocalidades[localidades.index(loc_suggested)]
        return [loc_suggested , idloc_suggested]
    except Exception as e:
        #print("Este erro",e)
        loc_suggested = " "
        idloc_suggested = " "
        return [loc_suggested , idloc_suggested]

In [3]:
def search_from_API(street,loc,dep,warning=False):  #Mejorar la funcion si no tiene localidad
    """Encuentra una localidad dada una calle sugerida y una localidad. Devuelve un diccionario con los datos de esa calle"""
    street, number = process_street(street)
    if street.isdigit():
        street += " " + street # La API busca los numeros de esta forma, ej: "60 60" es una busqueda valida para la calle 60
    suggestions = sugerir_calles(street)  #Consulta por calles similares a la API IDE
    try:
        street = str(street) #Transformo las calles de numeros en strings #transformo las 
    except Exception as e:
        return {}
    street = street.strip()   #Quito los espacios en blanco
    dep = dep.strip()
    if not suggestions: #Si no se devuelven calles similares
        if warning:
            print('No se encuentra calle simular')
        return  {}
    streets = [x.get('calle').lower() for x in suggestions] # Hay que trabajar con lower dado que las mayusculas afectan el ratio
    
    suggested_loc = [x.get('localidad').lower() for x in suggestions if x.get('localidad') is not None]
    suggested_idstreet = [x.get('idCalle') for x in suggestions] 
    suggested_idloc = [x.get('idLocalidad') for x in suggestions if x.get('idLocalidad') is not None] 
    suggested_dep = [x.get('departamento').lower() for x in suggestions if x.get('departamento') is not None] 
    suggested_streets = get_close_matches(street.lower(),street,n=14,cutoff=0.6) 
    try:
        loc, idloc = sugerir_localidad(loc.lower(),dep)
    except:
        return {}
    if not loc:
        if warning:
            print("no se encuentra localidad")
        return {}
    
    suggested_street_indexes = [x for x in range(len(suggested_loc)) if suggested_loc[x] == loc]
    suggested_streets = [streets[x] for x in suggested_street_indexes]
    suggested_idstreet = [suggested_idstreet[x] for x in suggested_street_indexes]
    suggested_dep =  [suggested_dep[x] for x in suggested_street_indexes]
    suggested_idloc = [suggested_idloc[x] for x in suggested_street_indexes]
   
    
    if not suggested_streets:
        if warning:
            print('No se encuentra calle similar')
        return {}
    else:
        try:
            result = ({'calle':suggested_streets[0], #Solo se informa el primero
                       'numero': number,
                   'id':suggested_idstreet[0],
                   'localidad':loc,
                   'idlocalidad':idloc,
                   'departamento':suggested_dep[0],
                   'fuente':'API INE'})
            return result
        except Exception as e:
            print(e, "FINAL")
            return {}


In [4]:
start = datetime.now()
calle = 'Salto' #8908
localidad = 'Montevideo'
departamento = 'MONTEVIDEO'
print("Sugerir calle dado la entrada {} en la localidad {}".format(calle,localidad))
print(search_from_API(calle,localidad,departamento).get('id'))
print("El script demoro:", datetime.now() - start)

Sugerir calle dado la entrada Salto en la localidad Montevideo


NameError: name 'process_street' is not defined

In [5]:
def search_from_frame(street,loc,dep,df):
    "Hace una busqueda en en cascada dep/loc/calle devolviendo un diccionario de datos"
    dep = dep.upper() #Pasa a mayusculas ya que esta asi en el DF 
    
    street, number = process_street(street)
    try:
        suggested_dep = get_close_matches(dep,df.departamento.unique(),n=14,cutoff=0.6)[0] #Prueba buscar el departamento
    except: # No se encuentra departamento
        return {}
    try:
        suggested_loc = get_close_matches(loc,df[df['departamento']==suggested_dep]['localidad'].unique(),n=14,cutoff=0.6)[0]
        
    except Exception as e:
        #print("No encuentro la localidad")
        return {} #No se encuentra la localidad
    try:
        suggested_street = get_close_matches(street,df[(df['departamento']==suggested_dep)&(df['localidad']==suggested_loc)]['calle'].unique(),n=10,cutoff=0.6)[0]
    except Exception as e:
        try: #Si la distancia de edición no funciona cuento cuantas veces aparecen los nombres de las palabras dentro de la calle en los candidatos. Quien tiene más palabras gana
            candidates = df[(df['departamento']==suggested_dep)&(df['localidad']==suggested_loc)]['calle'].unique()
            word_counts = [count_matches(street,candidate) for candidate in candidates]
            suggested_street = candidates[np.argmax(word_counts)]
        except:
            return {}
        
    try:
        _ , idloc = sugerir_localidad(suggested_loc,suggested_dep,limit=10)
        result = ({'calle':suggested_street , #Solo se informa el primero
                   'numero': number,
                   'id': int(df[(df['departamento']==suggested_dep)&(df['calle']==suggested_street)&(df['localidad']==suggested_loc)]['idcalle']),
                   'localidad':suggested_loc,
                   'idlocalidad':idloc,
                   'departamento':suggested_dep,
                   'fuente':'Codificador'})
        return result
    except:
        return {}


In [6]:
def count_matches(string1,string2):
    "Cuenta cuantas veces aparecen las palabras de una calle en otra"
    return sum([string2.count(x) for x in string1.split()])

In [7]:
#def process_city_to_list(loc):
#    "Procesa una strin que supuestamente es una localidad"
#    try:
#        return 
#    except:
#        return loc.strip()

In [8]:
def search_street(street,loc,dep,df):
    special_charecters = [",","-"]
    possible_locs = []
    for char in special_charecters: #Search for only one of this characters. 
        if char in loc:
            possible_locs = loc.split(char)
            break
    
    if isinstance(loc,str) :
        possible_locs.append(loc)
    else:
        possible_locs = [x.strip().upper() for x in possible_locs]
        
    if not dep:
        for loc in possible_locs:
            dep = suggest_dep(loc)
            if dep:
                possible_locs.pop(possible_locs.index(loc))
                break
         
    for loc in possible_locs:
        result = search_from_API(street,dep,loc)
        if result:
            return result
        else:
            result = search_from_frame(street,loc,dep,df)
            if result:
                return result  
            # Continua al proximo juego de palabras de la lista
    return {} # Si no encuentra match devuelve vacio

In [9]:
calle = "Salto 1500"
localidad = "montevideo ,montevideo"
departamento = ""
search_street(calle, localidad, departamento,df)

NameError: name 'df' is not defined

In [10]:
df_prueba1 = pd.read_csv(path_prueba1,sep=";",encoding = "ISO-8859-1")
print("Antes de limpiar el dataframe tiene {} filas y {} columas".format(df_prueba1.shape[0],df_prueba1.shape[1]))
df_prueba1 = df_prueba1[~df_prueba1['departamento'].isnull()] 
df_prueba1.reset_index(inplace=True)
df_prueba1.drop(columns=['index'],inplace=True)
print("Dsp de limpiar el dataframe tiene {} filas y {} columas".format(df_prueba1.shape[0],df_prueba1.shape[1]))
df_prueba1.head()

Antes de limpiar el dataframe tiene 5655 filas y 25 columas
Dsp de limpiar el dataframe tiene 4432 filas y 25 columas


,ID,depto,departamento,localidadbarrio,localidad,calle,esquina,npuerta,napartamento,manzana,...,ruta_km,idCalle,idPunto,puntoX,puntoY,puntoX_esquina,puntoY_esquina,puntoX_ruta,puntoY_ruta,"punto_esquina,,,,,,"
0,1,1.0,MONTEVIDEO ...,LA COMERCIAL,MONTEVIDEO,NUEVA PALMIRA,M.C.MARTINEZ,1939,999,999,...,NaN,9597.0,478070.0,-561.738.767.479.425,-3.488.858.993.825.970,,,,,",,,,,,"
1,3,1.0,MONTEVIDEO ...,La Comercial,MONTEVIDEO,Justicia,Libres,2382,4,999,...,NaN,9817.0,494161.0,-56.179.405.115.125.900,-3.488.620.840.614.770,,,,,",,,,,,"
2,4,1.0,MONTEVIDEO ...,Aguada,MONTEVIDEO,Estrella del Norte,Batoví,1817,2,999,...,NaN,9762.0,490139.0,-56.182.100.352.352.000,-3.489.477.333.235.400,,,,,",,,,,,"
3,7,1.0,MONTEVIDEO ...,Villa Muñoz,MONTEVIDEO,Domingo Aramburú,Arenal Grande,1816,999,999,...,NaN,9318.0,452056.0,-5.617.830.009.800.110,-3.488.380.937.643.790,,,,,",,,,,,"
4,8,1.0,MONTEVIDEO ...,Malvín Norte,MONTEVIDEO,José de Freitas,Gallinal,4876,999,999,...,NaN,13089.0,2366666.0,-5.611.022.134.824.960,-34.871.969.169.061.800,,,,,",,,,,,"


In [11]:
def check_results_from_data(columna_calle,columna_localidad,columna_departamento,frame,referencia,df_append):
    "Analiza un frame para buscar calles"
    dep_error = []
    loc_error = []
    calle_error = []
    start = datetime.now()
    start_index = ""
    for index, row in frame.iterrows():
        if not start_index:
            start_index = index
        if index%1000 == 0:
            #d = {'calle':calle_error, 'localidad':loc_error,'departamento':dep_error}
            #inter = pd.DataFrame(data = d)
            #df_append.append(inter)
            print("Van {} y faltan {}, van {} errores".format(index,frame.shape[0]-index+start_index,len(dep_error)))
            print("Van {} minutos".format((datetime.now() - start).seconds/60))
        calle = row[columna_calle]
        localidad =  row[columna_localidad].strip()
        departamento = row[columna_departamento].strip()
        
        result = search_street(calle,localidad,departamento,referencia)
        if not result:
            dep_error.append(departamento)
            loc_error.append(localidad)
            calle_error.append(calle)
    d = {'calle':calle_error, 'localidad':loc_error,'departamento':dep_error}
    inter = pd.DataFrame(data = d)
    df_append = df_append.append(inter)
    return df_append

In [12]:
def check_IDE_dataset(df_prueba1):
    dep_error = []
    loc_error = []
    calle_error = []
    id_error = []
    tipo_error = []

    for index, row in df_prueba1.iterrows():
        if index%1000 == 0:
            print("Van {} y faltan {}, van {} errores".format(index,df_prueba1.shape[0]-index,len(dep_error)))

        calle = str(row['calle'])
        localidad =  row['localidad'].strip()
        departamento = row['departamento'].strip()

        try:
            id_calle = int(row['idCalle'])
        except:
            id_calle = np.nan

      #Da 500 ms de espera para que la API no me bloquee
       #  try: # Prueba si la calle es valida
       #    if int(row[columna_calle]):
       #         dep_error.append(row[columna_departamento].strip())
       #         loc_error.append(row[columna_localidad].strip())
       #         calle_error.append(row[columna_calle])
       #         #id_error.append(row['idCalle']) #Solo para df de INE
       #          tipo_error.append("La calle no es valida")
       # except:
       #     None

        result = search_street(calle,localidad,departamento,df)
        #print(index,result)
        if not result:
            dep_error.append(departamento.strip())
            loc_error.append(localidad.strip())
            calle_error.append(calle)
            id_error.append(id_calle) 
            tipo_error.append("Resultado vacio")
            #print("aca",result,row['calle'])
            continue
        try:     
            if  result.get('id') != id_calle and not np.isnan(id_calle):
                dep_error.append(departamento.strip())
                loc_error.append(localidad.strip())
                calle_error.append(calle)
                id_error.append(id_calle)
                tipo_error.append("Difieren ids")
                continue

                #print(row['departamento'].strip(),row['localidad'.strip()],row['calle'].strip())
        except Exception as e:
            dep_error.append(departamento.strip())
            loc_error.append(localidad.strip())
            calle_error.append(calle)
            id_error.append(id_calle)
            tipo_error.append(e)
            print("Hubo un error",e,departamento,localidad,id_calle)

    # Genera el dataframe con los errores
    d = {'calle':calle_error,'id':id_error, 'localidad':loc_error,'departamento':dep_error,'tipo error':tipo_error}
    error_API_DF = pd.DataFrame(data = d) 
    error_API_DF.to_csv('errores buscando por API y DF.csv',index=False)
    print("Archivo generado con éxito, se detactaron {} errores".format(len(calle_error)))
    print("El script demoró {} minutos".format((datetime.now()-start).minutes)/60)
    return error_API_DF

In [13]:
df_errores = pd.read_csv("errores buscando por API y DF.csv")

In [15]:
del df_errores

Salvo la calle ciento uno todos los errores vienen por duplicados

In [ ]:
# Para checkear el dataset de SIIAS

#df_prueba2 = pd.read_csv(path_prueba2,encoding = "ISO-8859-1", error_bad_lines=False)
##a_borrar = 14000
#df_prueba2.drop(range(a_borrar),inplace=True)
#A = df_prueba2.loc[270000:272000]

In [ ]:
#Lista de errores encontrados en sias /

table = [["calle","localidad","departamento"],
['','','MONTEVIDEO'],
['DOS','MONTEVIDEO','MONTEVIDEO'],
['TREINTA Y TRES','MONTEVIDEO','MONTEVIDEO'],
['SBRE DE PASO','MELILLA','MONTEVIDEO'],
['CAMINO AL PASO HONDO',"","MONTEVIDEO"]]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

In [ ]:
months = (["ENERO",
               "FEBRERO",
               "MARZO",
               "ABRIL",
               "MAYO",
               "JUNIO",
               "JULIO",
               "AGOSTO",
               "SETIEMBRE", #SEPTIEMBRE TMB?
               "OCTUBRE",
               "NOVIEMBRE",
               "DICIEMBRE"]) 

In [ ]:
def number_in_text_to_digit(street):
    "Checks if a number is in text and changes to digits"

    correct_number = {'PRIMERO':1,
                      'SEGUNDO':2,
                      'TRES':3,
                      'CUATRO':4,
                      'CINCO':5,
                      'SEIS':6,
                      'SIETE':7,
                      'OCHO':8,
                      'NUEVE':9,
                      'DIEZ':10,
                      'ONCE':11,
                      'DOCE':12,
                      'TRECE':13,
                      'CATORCE':14,
                      'QUINCE':15,
                      'DIECISEIS':16,
                      'DIECISIETE':17,
                      'DIECIOCHO':18,
                      'DIECINUEVE':19,
                      'VEINTE':20,
                      'VEINTIUNO':21,
                      'VEINTIDOS':22,
                      'VEINTITRES':23,
                      'VEINTICUATRO':24,
                      'VEINTICINCO':25,
                      'VEINTISEIS':26,
                      'VEINTISIETE':27,
                      'VEINTIOCHO':28,
                      'VEINTINUEVE':29,
                      'TREINTA':30,
                      'TREINTA Y UNO':31
                     }
    number = list(set(correct_number.keys())&set(street.split())
    return street.replace(number,correct_number.get(number))
    

In [23]:
loc_INE = pd.read_csv(localidades_INE,sep=";",encoding = "ISO-8859-1")

In [25]:
loc_activas = loc_INE[loc_INE['INEIGLOCACTIVA']=='S']
loc_activas

,,INEIGDEPCODIGO,INEIGLOCCODIGO,INEIGLOCNOMBRE,INEIGLOCACTIVA
2,3,05,931,Blanca Arena,S
3,4,05,932,Brisas del Plata,S
9,10,05,938,Colonia Cosmopolita,S
11,12,05,940,Paraje Minuano,S
21,22,03,326,Haras del Lago,S
...,...,...,...,...,...
1227,1228,09,721,Pirarajá,S
1228,1229,09,722,Zapicán,S
1229,1230,09,821,Colón,S
1231,1232,09,823,Aramendía,S


In [27]:
dire_INE = pd.read_csv(base_INE,sep=";",encoding = "ISO-8859-1")

In [30]:
dire_INE.columns

Index(['   ', 'INEMDDEPARTAMENTO', 'INEMDLOCALIDAD', 'INEMDNOMCALLE',
       'INEMDCODCALLE', 'INEMDNROPUERTA'],
      dtype='object')

In [48]:
loc_activas['COD'] = loc_activas['INEIGDEPCODIGO'] + loc_activas['INEIGLOCCODIGO']

e:\python\ine\codificacion_calles\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
dicti = loc_activas.set_index('INEIGLOCCODIGO')['INEIGLOCNOMBRE'].to_dict()

In [44]:
loc_activas.set_index('INEIGLOCCODIGO')

,,INEIGDEPCODIGO,INEIGLOCNOMBRE,INEIGLOCACTIVA
INEIGLOCCODIGO,,,,
931,3,05,Blanca Arena,S
932,4,05,Brisas del Plata,S
938,10,05,Colonia Cosmopolita,S
940,12,05,Paraje Minuano,S
326,22,03,Haras del Lago,S
...,...,...,...,...
721,1228,09,Pirarajá,S
722,1229,09,Zapicán,S
821,1230,09,Colón,S


In [47]:
len(loc_activas['INEIGLOCCODIGO'].unique())

169